## Load and prepare data 

Load and prepare data for backtrajectory calculations.

- Load the NSIDC PP data: *D*

    - Loading a single, concatenated dataset from multiple source files.
    - Interpolating over a few holes in the data.
    
- Load a land mask: *MASK*
    - Define a land mask of land/no land that we use to kill trajectories that intersect a coastline.

In [1]:
import numpy as np
import xarray as xr
import glob
import pyproj
import matplotlib.pyplot as plt
from matplotlib.dates import date2num

/home/oyvindl/mambaforge/envs/oyv/lib/python3.12/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


### Load data

Replace the *ddir* folder with the directory containing the PP NSIDC files (e.g. *icemotion_daily_nh_25km_20120101_20121231_v4.1.nc*)

In [2]:
ddir ='/home/oyvindl/work/data/external/sea_ice/sea_ice_drift/nsidc_pp_v4/'
flist = glob.glob(ddir + '*nc')

Lazy load of the data (concatenating the files) 

In [3]:
# Loading
# ("minimal" keyword avoids adding a time dimension to lat,lon)
try:
    D = xr.open_mfdataset(flist, data_vars = 'minimal')
except:
    raise Exception('Failed to open files..\n(Looked for files here: %s)'%ddir)
# Last value is empty -> drop
D = D.drop_sel(time=D.time.data[-1])

### Interpolate over NaN days

Some days, like 31.12 on 2020 and 2021, are all nans. 

-> Linearly interpolating between adjectent days (considering NaNs as zeros, 
but setting all-zero points to nan..)

In [4]:
# Indices where all values are NaN
isnan_all = np.isnan(D.u.max(dim = ['x', 'y']))
is_nan_indices = np.where(isnan_all)[0]

In [5]:
# For each Nan index: Interpolate across from adjacent days

for nan_ind in is_nan_indices:
    u_nanind =  0.5*(D.u[nan_ind-1].fillna(0) + D.u[nan_ind+1].fillna(0))
    v_nanind =  0.5*(D.v[nan_ind-1].fillna(0) + D.v[nan_ind+1].fillna(0))
    
    u_nanind = u_nanind.where(np.bool_((u_nanind!=0) * (v_nanind!=0)))
    v_nanind = v_nanind.where(np.bool_((u_nanind!=0) * (v_nanind!=0)))

    D.u[nan_ind] = u_nanind
    D.v[nan_ind] = v_nanind

##### (Optional): Quick check

Toggle *check_interpolation=True* to show a quick check of the interpolation on Dec 31 2020..


In [6]:
check_interpolation = False

if check_interpolation:
    x_point, y_point = 180, 180
    
    fig, ax = plt.subplots(figsize = (11, 3))
    
    ax.plot_date(date2num(D.time), D.u.isel(x=x_point, y=y_point), '-k', alpha = 0.3)
    ax.plot_date(date2num(D.time), D.u.isel(x=x_point, y=y_point), '.', ms = 6, 
                 label = 'u (original)')
    ax.plot_date(D.time[is_nan_indices[0]], 
                 D.u.isel(x=x_point, y=y_point)[is_nan_indices[0]], '*y', ms = 12,
                 label = 'u (interpolated)')
    ax.plot_date(date2num(D.time), D.v.isel(x=x_point, y=y_point), '-k', alpha = 0.3)
    ax.plot_date(date2num(D.time), D.v.isel(x=x_point, y=y_point), '.', ms = 6, 
                 label = 'v (original)')
    ax.plot_date(D.time[is_nan_indices[0]], 
                 D.v.isel(x=x_point, y=y_point)[is_nan_indices[0]], '*r', ms = 12, 
                 label = 'v (interpolated)')

    for t_ in date2num(D.time):
        ax.vlines(t_, -50, 50, 'k', alpha = 0.2)
    ax.set_xlim(18618, 18633)
    ax.set_ylim(-10, 10)
    ax.legend(ncol = 2, fontsize = 9)

### Define a land mask

Here, we can define a land mask (*MASK*) that will be used to kill a backtrajectory when it intersects a coastline.

*MASK* should have the following variables:
- *x, y* - Grid 
- *is_land* - Boolean (land=1, not land = 0)  
- *proj* - pyproj.CRS defining the grid projection of *x, y*


In my case, I am using a 2km land mask for the Eurasian sector based on IBCAO-v4, where I have removed Svalbard in order to allow trajectories to pass through islands. This mask was made elsewhere - I am just loading it here.

We can choose not to use a land mask. In that case, set *no_land_mask = True*

In [7]:
no_land_mask = False

In [8]:
# Load the land mask based on IBCAO (and defining the projection of this mask):
if not no_land_mask:
    my_mask_proj = pyproj.CRS('epsg:3996')  
    my_mask = xr.open_dataset(
        '/home/oyvindl/work/data/external/bathymetry/ibcao_v4/netcdf/IBCAO_landmask_greater_barents_2km.nc')
    mask_is_land = my_mask.landmask_wo_svb
    # (Also adding lon/lat, but thise aren't strictly necessary)
    MASK = xr.Dataset(coords = {'x':my_mask.x, 'y':my_mask.y},
                     data_vars = {'lon':my_mask.lon, 'lat':my_mask.lat,
                                  'is_land':mask_is_land})
    MASK['proj'] = ((), my_mask_proj)

CRSError: Invalid projection: EPSG:3996: (Internal Proj Error: proj_create: no database context specified)

In [14]:
# If we dont have a land mask/dont want to use one:
if no_land_mask:
    MASK = None

##### (Optional): Quick check of mask

Toggle *show_mask=True* to show a quick landmask plot

In [45]:
show_mask = False

if show_mask:

    import cartopy.crs as ccrs
    import cartopy.feature as cfeature

    lat_ctr, lon_ctr = 86, 30

    cart_proj = ccrs.Stereographic(
        central_latitude=lat_ctr,
        central_longitude=lon_ctr,
        scale_factor=None, globe=None)

    fig = plt.figure(figsize = (8, 8))
    ax = plt.axes(projection=cart_proj)

    # Coastline
    ax.coastlines('50m', )

    # Land
    land_50m = cfeature.NaturalEarthFeature(
        'physical', 'land', '50m',
        edgecolor='k', linewidth = 1,
        facecolor='none')

    ax.add_feature(land_50m)

    # Set boundaries
    ax.set_extent([-20, 191, 66, 89.5], crs=ccrs.PlateCarree())
    
    # Transform mask coordiates (x-y) to (lat-lon) 
    wgs84_proj = pyproj.CRS("EPSG:4326") # latlon with WGS84 datum
    MASK_to_latlon = pyproj.Transformer.from_crs(MASK.proj.data.item(), wgs84_proj)
    mask_lon, mask_lat = None, None
    X, Y = np.meshgrid(MASK.x, MASK.y)
    LAT, LON = MASK_to_latlon.transform(X, Y)
    ax.pcolormesh(LON, LAT, MASK.is_land.values, transform=ccrs.PlateCarree())